# Kullback-Leibler divergence

Use K-L divergence to determine the effect of target variable (activity) values (0 or 1) on a Gaussian mixture model over the data.

Apply this only to the continuous features, separately for cid and pid.

References:

* [KL Divergence Python Example](https://towardsdatascience.com/kl-divergence-python-example-b87069e4b810)—a nice explanation and example
* [Stackexchange: Calculating KL Divergence in Python
](https://datascience.stackexchange.com/questions/9262/calculating-kl-divergence-in-python)—some good notes on which Python libraries to use
* [Mutual Information](https://en.wikipedia.org/wiki/Mutual_information)—seems relevant here
* [Sensitivity Analysis in Gaussian Bayesian Networks Using a Divergence Measure](https://www.researchgate.net/publication/233216409_Sensitivity_Analysis_in_Gaussian_Bayesian_Networks_Using_a_Divergence_Measure)
* [Clustering with Gaussian Mixture Models](https://pythonmachinelearning.pro/clustering-with-gaussian-mixture-models/)
* [scikit-learn Gaussian mixture models](https://scikit-learn.org/stable/modules/mixture.html)
* [Approximating the Kullback Leibler Divergence Between Gaussian Mixture Models](https://www.researchgate.net/publication/4249249_Approximating_the_Kullback_Leibler_Divergence_Between_Gaussian_Mixture_Models)
* [Stackoverlflow: KL-Divergence of two GMMs
](https://stackoverflow.com/questions/26079881/kl-divergence-of-two-gmms)
* [Stackexchange: Trying to implement the Jensen-Shannon Divergence for Multivariate Gaussians](https://stats.stackexchange.com/questions/345915/trying-to-implement-the-jensen-shannon-divergence-for-multivariate-gaussians)—related and refering to the above Stackoverflow Q&A
* [Stackoverlflow: predict_proba is not working for my gaussian mixture model (sklearn, python)
](https://stackoverflow.com/questions/56993070/predict-proba-is-not-working-for-my-gaussian-mixture-model-sklearn-python)

In [81]:
from sklearn.mixture import GaussianMixture
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

### Load the features files for CIDs and PIDs

In [2]:
data_loc = '../data/FDA-COVID19_files_v1.0/'

In [3]:
# Get the individual feature sets as data frames
  def __load_feature_files():
    print('===============================================')
    print('dragon_features.csv')
    print('===============================================')
    # note need to set the data_type to object because it complains, otherwise that the types vary.
    df_dragon_features = __load_data(data_loc+'drug_features/dragon_features.csv', data_type=object)
    
    # rename the dragon features since there are duplicate column names in the protein binding-sites data.
    df_dragon_features.columns = ['cid_'+col for col in df_dragon_features.columns]
    
    # handle na values in dragon_features
    # Many cells contain "na" values. Find the columns that contain 2% or 
    # less of these values and retain them, throwing away the rest. 
    # Then mean-impute the "na" values in the remaining columns.
    pct_threshold = 2
    na_threshold = int(len(df_dragon_features)*pct_threshold/100)
    ok_cols = []

    for col in df_dragon_features:
        na_count = df_dragon_features[col].value_counts().get('na')
        if (na_count or 0) <= na_threshold:
            ok_cols.append(col)

    print('number of columns where the frequency of "na" values is <= {}%: {}.'.format(pct_threshold, len(ok_cols)))
    
    df_dragon_features = df_dragon_features[ok_cols]

    # convert all values except "na"s to numbers and set "na" values to NaNs.
    df_dragon_features = df_dragon_features.apply(pd.to_numeric, errors='coerce')

    columns_missing_values = df_dragon_features.columns[df_dragon_features.isnull().any()].tolist()
    print('{} columns with missing values.'.format(len(columns_missing_values)))

    # replace NaNs with column means
    df_dragon_features.fillna(df_dragon_features.mean(), inplace=True)

    columns_missing_values = df_dragon_features.columns[df_dragon_features.isnull().any()].tolist()
    print('{} columns with missing values (after imputing): {}'.format(len(columns_missing_values), 
                                                                       columns_missing_values))    
    print('===============================================')
    print('binding_site_features_v2.csv')
    print('===============================================')
    df_binding_sites = __load_data(data_loc+'protein_features/binding_site_features_v2.csv')
    
    # Name the index to 'pid' to allow joining to other feaure files later.
    df_binding_sites.index.name = 'pid'
    
    print('===============================================')
    print('expasy.csv')
    print('===============================================')
    df_expasy = __load_data(data_loc+'protein_features/expasy.csv')
    
    print('===============================================')
    print('profeat.csv')
    print('===============================================')
    df_profeat = __load_data(data_loc+'protein_features/profeat.csv')
    
    # Name the index to 'pid' to allow joining to other feaure files later.
    df_profeat.index.name = 'pid'
    
    # profeat has some missing values.
    s = df_profeat.isnull().sum(axis = 0)

    print('number of missing values for each column containing them is: {}'.format(len(s[s > 0])))

    # Drop the rows that have missing values.
    df_profeat.dropna(inplace=True)
    print('number of rows remaining, without NaNs: {:,}'.format(len(df_profeat)))
    
    return {'df_dragon_features': df_dragon_features,
           'df_binding_sites': df_binding_sites,
           'df_expasy': df_expasy,
           'df_profeat': df_profeat}

In [4]:
# load a specific features CSV file
  def __load_data(path, data_type=None):
    if data_type:
        df = pd.read_csv(path, index_col=0, dtype=data_type)
    else:
        df = pd.read_csv(path, index_col=0)
    print('Number of rows: {:,}'.format(len(df)))
    print('Number of columns: {:,}'.format(len(df.columns)))
    
    columns_missing_values = df.columns[df.isnull().any()].tolist()
    print('{} columns with missing values'.format(len(columns_missing_values)))
    
    print(df.head(2))
    
    return df

In [5]:
feature_sets = __load_feature_files()

df_dragon_features = feature_sets['df_dragon_features']
df_binding_sites = feature_sets['df_binding_sites']
df_expasy = feature_sets['df_expasy']
df_profeat = feature_sets['df_profeat']

dragon_features.csv
Number of rows: 88,105
Number of columns: 3,839
0 columns with missing values
              MW                AMW      Sv                  Se  \
cid                                                               
72792562  474.67  6.781000000000001  41.039              70.101   
44394609  546.48              8.674  43.185  63.538000000000004   

                          Sp                 Si     Mv                  Me  \
cid                                                                          
72792562   43.54600000000001  80.52199999999999  0.586               1.001   
44394609  45.233000000000004             69.993  0.685  1.0090000000000001   

             Mp     Mi  ... Psychotic-80 Psychotic-50 Hypertens-80  \
cid                     ...                                          
72792562  0.622   1.15  ...            0            0            0   
44394609  0.718  1.111  ...            0            0            0   

         Hypertens-50 Hypnotic-80 Hypno

### Merge the features for CIDs and PIDs with the interactions.

This yields a set of CID features and a set of PID features.

In [30]:
interactions = '../data/validation_interactions_v5.csv'
df_interactions = __load_data(interactions)

df_pid = pd.merge(df_interactions, df_expasy, on='pid', how='inner')
df_pid = pd.merge(df_pid, df_profeat, on='pid', how='inner')

df_cid = pd.merge(df_interactions, df_dragon_features, on='cid', how='inner')

Number of rows: 7,972
Number of columns: 5
0 columns with missing values
    cid       pid  activity  cid_binary_weights  pid_binary_weights
0   938  AAB59829         1            0.952894            0.996703
1  1986  AAB59829         1            0.975912            0.996703


### Check for types and missing values

In [42]:
df_pid.activity = df_pid.activity.astype(float)
df_cid.activity = df_cid.activity.astype(float)
print(df_pid.shape)
print(df_cid.shape)

(7763, 861)
(7969, 3645)


In [43]:
# Any missing values?
print(df_pid.isnull().values.any())
print(df_cid.isnull().values.any())

False
False


In [37]:
df_pid.dtypes.value_counts()

float64    859
int64        1
object       1
dtype: int64

In [38]:
df_cid.dtypes.value_counts()

float64    3597
int64        47
object        1
dtype: int64

In [40]:
df_cid.columns.to_series().groupby(df_cid.dtypes).groups

{dtype('int64'): Index(['cid', 'cid_nAT', 'cid_nSK', 'cid_nTA', 'cid_nBT', 'cid_nBO', 'cid_nBM',
        'cid_RBN', 'cid_nDB', 'cid_nTB', 'cid_nAB', 'cid_nH', 'cid_nC',
        'cid_nN', 'cid_nO', 'cid_nP', 'cid_nS', 'cid_nF', 'cid_nCL', 'cid_nBR',
        'cid_nI', 'cid_nB', 'cid_nHM', 'cid_nHet', 'cid_nX', 'cid_nCsp3',
        'cid_nCsp2', 'cid_nCsp', 'cid_nStructures', 'cid_totalcharge',
        'cid_nCIC', 'cid_nCIR', 'cid_TRS', 'cid_Rperim', 'cid_Rbrid', 'cid_NRS',
        'cid_nR03', 'cid_nR04', 'cid_nR05', 'cid_nR06', 'cid_nR07', 'cid_nR08',
        'cid_nR09', 'cid_nR10', 'cid_nR11', 'cid_nR12', 'cid_nBnz'],
       dtype='object'),
 dtype('float64'): Index(['activity', 'cid_binary_weights', 'pid_binary_weights', 'cid_MW',
        'cid_AMW', 'cid_Sv', 'cid_Se', 'cid_Sp', 'cid_Si', 'cid_Mv',
        ...
        'cid_Hy', 'cid_TPSA(NO)', 'cid_TPSA(Tot)', 'cid_SAtot', 'cid_SAacc',
        'cid_SAdon', 'cid_Vx', 'cid_VvdwMG', 'cid_VvdwZAZ', 'cid_PDI'],
       dtype='object', length=

In [41]:
df_pid.head()

,cid,pid,activity,cid_binary_weights,pid_binary_weights,helical,beta,coil,veryBuried,veryExposed,...,[G7.1.1.71],[G7.1.1.72],[G7.1.1.73],[G7.1.1.74],[G7.1.1.75],[G7.1.1.76],[G7.1.1.77],[G7.1.1.78],[G7.1.1.79],[G7.1.1.80]
0,938,AAB59829,1.0,0.952894,0.996703,0.349,0.219,0.432,0.326,0.187,...,0.003074,0.000574,-0.001384,-0.001869,-0.000164,-0.001509,0.000604,-0.000104,0.001366,0.002322
1,1986,AAB59829,1.0,0.975912,0.996703,0.349,0.219,0.432,0.326,0.187,...,0.003074,0.000574,-0.001384,-0.001869,-0.000164,-0.001509,0.000604,-0.000104,0.001366,0.002322
2,37542,AAB59829,0.0,0.963498,0.996703,0.349,0.219,0.432,0.326,0.187,...,0.003074,0.000574,-0.001384,-0.001869,-0.000164,-0.001509,0.000604,-0.000104,0.001366,0.002322
3,445580,AAB59829,0.0,0.973330,0.996703,0.349,0.219,0.432,0.326,0.187,...,0.003074,0.000574,-0.001384,-0.001869,-0.000164,-0.001509,0.000604,-0.000104,0.001366,0.002322
4,4100,AAB59829,0.0,0.923492,0.996703,0.349,0.219,0.432,0.326,0.187,...,0.003074,0.000574,-0.001384,-0.001869,-0.000164,-0.001509,0.000604,-0.000104,0.001366,0.002322


### Train a GMM over each feature set

In [186]:
gmm_pid = GaussianMixture(n_components=100,
              covariance_type='full', max_iter=20, random_state=42, reg_covar=0.00001)

gmm_cid = GaussianMixture(n_components=100,
              covariance_type='full', max_iter=10, random_state=42, reg_covar=0.00001)

# Drop non-feature cols
# Leave activity in there we'll flip that to determine sensistivity
pids = df_pid.drop(['cid','pid','cid_binary_weights','pid_binary_weights'], axis=1)
cids = df_cid.drop(['cid','pid','cid_binary_weights','pid_binary_weights'], axis=1)

In [187]:
pid_scaler = MinMaxScaler()
pids = pid_scaler.fit_transform(pids)
cid_scaler = MinMaxScaler()
cids = cid_scaler.fit_transform(cids)

In [ ]:
gmm_cid.fit(cids)
gmm_cid.converged_

In [189]:
gmm_pid.fit(pids)
gmm_pid.converged_

True

### Mahalanobis distance differences when flipping activity

For each point, obtain a prediction from GMM of its closest cluster. Then calculate the mahalanobis distance to its  cluster. Next, flip the activity bit and re-predict and calculate distance again. Finally, calculate the poiint's score given the change in distance. NOTE: this assumes that flipping activity does not change cluster membership, only likelihood of membership of the same cluster.

In [190]:
from scipy.spatial import distance

def dist(model, v):
    p = model.predict_proba(v)
    cluster = np.argmax(p)
    mu = model.means_[cluster]
    cov = model.covariances_[cluster]
    icov = np.linalg.inv(cov) 
    return distance.mahalanobis(v, mu, icov)

def get_score(vin, model):
    v = [vin]
    d1 = dist(model, v)
    print(d1)

    # Flip activity and measure distance
    if vin[0] == 1.0:
        vin[0] = 0.0
    else:
        vin[0] = 1.0

    d2 = dist(model, v)
    print(d2)
    
    score = 1 - 2 * np.abs(d1-d2)/d1+d2
    return score
    
pid_scores = [get_score(v, gmm_pid) for v in pids[:10]] # Try on the first 10 pids

3.2657779669195013
0.30616668439870326
3.2657779669195013
0.30616668439870326
0.30616668439870326
3.2657779669195013
0.30616668439870326
3.2657779669195013
0.30616668439870326
3.2657779669195013
0.30616668439870326
3.2657779669195013
0.30616668439870326
3.2657779669195013
0.30616668439870326
3.2657779669195013
0.30616668439870326
3.2657779669195013
0.30616668439870326
3.2657779669195013


In [191]:
pid_scores

[-0.5063333156012968,
 -0.5063333156012968,
 -15.067555366413831,
 -15.067555366413831,
 -15.067555366413831,
 -15.067555366413831,
 -15.067555366413831,
 -15.067555366413831,
 -15.067555366413831,
 -15.067555366413831]

### For each row in each feature set flip the activity and measure the difference in class membership

In [78]:
gmm_pid.predict_proba(pid_scaler.transform([pids[0]]))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

### K-L divergence

Since there is no closed-form solution over GMMs we will use an approximation based upon Monte Carlo simulation.

In [ ]:
def gmm_kl(gmm_p, gmm_q, n_samples=10**5):
    X = gmm_p.sample(n_samples)
    log_p_X, _ = gmm_p.score_samples(X)
    log_q_X, _ = gmm_q.score_samples(X)
    return log_p_X.mean() - log_q_X.mean()

### For reference: Brian's code for doing a similiar thing on the binary features using LDA:

In [ ]:
# validation weighting
# train LDA for validation weights
validation_interactions = pd.read_csv(validation_interactions_csv, index_col=0)
validation_interactions.drop("latent_prob_delta_ratio", axis=1, inplace=True)
required_training_interactions = pd.read_csv(required_training_interactions_csv, index_col=0)
optional_training_interactions = pd.read_csv(optional_training_interactions_csv, index_col=0)

opt_unique = optional_training_interactions.drop_duplicates(["cid", "activity"]).drop("activity_score", axis=1)
req_unique = required_training_interactions.drop_duplicates(["cid", "activity"])

training_unique = pd.concat([opt_unique, req_unique]).drop_duplicates(["cid", "activity"])
num_topics = 100
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42, learning_method="online", n_jobs=-1)

cid_fingerprints_file = "dataset_raw_files/fingerprints.csv"
cid_fingerprints = pd.read_csv(cid_fingerprints_file)
cid_interactions = cid_fingerprints.merge(training_unique.drop("pid", axis=1), on="cid").drop("cid", axis=1)

lda.fit(cid_interactions)
del cid_interactions
cid_fingerprints["pos"] = 1
cid_fingerprints["neg"] = 0
cid_fingerprints.set_index("cid", inplace=True)

latent_prob_pos = np.max(lda.transform(cid_fingerprints.drop("neg", axis=1)), axis=1)
cid_fingerprints["latent_prob_neg"] = np.max(lda.transform(cid_fingerprints.drop("pos", axis=1)), axis=1)
cid_fingerprints.reset_index(inplace=True)
cid_fingerprints["latent_prob_pos"] = latent_prob_pos
cid_fingerprints.drop(["pos", "neg"], axis=1, inplace=True)
cid_fingerprints["latent_prob_delta"] = np.abs(cid_fingerprints.latent_prob_pos - cid_fingerprints.latent_prob_neg)
cid_fingerprints["latent_prob_delta_ratio"] = 1 - 2 * cid_fingerprints.latent_prob_delta / (cid_fingerprints.latent_prob_pos + cid_fingerprints.latent_prob_neg)